In [7]:
#Import train dataset
import pandas as pd

pd.read_csv('train.csv')

df_edges_all=pd.read_csv('train.csv')


df_edges= df_edges_all.iloc[:int(len(df_edges_all)*0.8)]

df_edges_test = df_edges_all.iloc[int(len(df_edges_all)*0.8):]
df_edges.head()


,id,id1,id2,label
0,1,9202,9202,1
1,2,410411,460254,0
2,3,211858,312074,1
3,6,253901,504325,0
4,7,415071,63239,0


In [8]:
df_edges_test.head()

,id,id1,id2,label
758585,976717,345657,467621,1
758586,976719,825394,20456,1
758587,976720,720678,447520,0
758588,976721,181991,649051,1
758589,976723,663566,821902,0


In [9]:
# The existing graph
import networkx as nx
G= nx.Graph()
#add nodes
nodes = pd.unique(df_edges_all[['id1', 'id2']].values.ravel())
G.add_nodes_from(nodes)
#add edges
edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==1].itertuples()]
G.add_edges_from(edges)

# keeping all the negative edges so we can compare results
no_edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==0].itertuples()]

In [19]:
import math
# Adding methods proposed in the https://link.springer.com/content/pdf/10.1038/s41598-019-57304-y.pdf
def shortest_path(G:nx.Graph,source:int,target:int)->float:
    try:
        p=nx.shortest_path(G,source=source,target=target)
        return len(p)-1
    except nx.NetworkXNoPath:
        return float("inf")

def distance(G:nx.Graph,source:int,target:int)->float:
   '''
   This method can be changed to any distance
   '''
   return shortest_path(G,source,target)

def distance_description(source:int,target:int)->float:
    '''
    This method can be changed to any distance
    '''
    return source-target


def closeness_centrality(G:nx.Graph, source:int, target:int)->float:
    dxy = nx.shortest_path(G,source=source,target=target)
    return G.number_of_nodes()/shortest_path(G,source,target)

def common_neighbors(G:nx.Graph, source:int, target:int)->list:
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    return common_neighbors

def CCPA(G:nx.Graph, source:int, target:int,a:float = 0.5)->float:
    '''
    Common Neighbor and Centrality based Parameterized Algorithm
    '''
    return a*closeness_centrality(G,source,target)+(1-a)*len(common_neighbors(G,source,target))

def CND(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor and Distance
    '''
    cn = common_neighbors(G,source,target)
    if len(cn)>0:
        return (len(cn)+1)/2
    else:
        return 1/distance(G,source,target)

def PA(G:nx.Graph, source:int, target:int)->float:
    '''
    Preferential Attachment
    '''
    return G.degree[source]*G.degree[target]

def AA(G:nx.Graph, source:int, target:int)->float:
    '''
    Adamic Adar
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / math.log(degree)
    return similarity

def CN(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor
    '''
    return len(common_neighbors(G,source,target))

def SI(G:nx.Graph, source:int, target:int)->float:
    '''
    Sorensen Index
    '''
    if (G.degree[source]+G.degree[target]) ==0:
        return 0
    return 2*CN(G,source,target)/(G.degree[source]+G.degree[target])

def JI(G:nx.Graph, source:int, target:int)->float:
    '''
    Jaccard Index
    '''
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    all_neighbors = set(s_neighbors).union(t_neighbors)
    if len(all_neighbors)==0:
        return 0
    return len(common_neighbors)/len(all_neighbors)

def RA(G:nx.Graph, source:int, target:int)->float:
    '''
    Resource Allocation
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / degree
    return similarity

def HPI(G:nx.Graph, source:int, target:int)->float:
    '''
    Hub Promoted Index
    '''
    if min([G.degree[source],G.degree[target]]) ==0 :
        return 0
    return CN(G,source,target)/min([G.degree[source],G.degree[target]])

def create_metric_vector(G: nx.Graph, source: int, target: int):
    functions = [CND, PA, AA, CN, SI, JI, RA, HPI]
    outputs = []
    for func in functions:
        output = func(G, source, target)
        outputs.append(output)
    return outputs




In [20]:
# metrics names
print("[CND, PA, AA, CN, SI, JI, RA, HPI]")
#This for connected nodes
print(create_metric_vector(G,211858,312074))
#This is for unconnected nodes
print(create_metric_vector(G,410411,460254))


[CND, PA, AA, CN, SI, JI, RA, HPI]
[1.0, 18, 0.0, 0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0, 0.0, 0, 0.0, 0.0, 0.0, 0]


In [21]:
# TEST SET

# metrics names
print("[CND, PA, AA, CN, SI, JI, RA, HPI]")
#This for connected nodes
print(create_metric_vector(G,345657,467621))
#This is for unconnected nodes
print(create_metric_vector(G,720678,447520))

[CND, PA, AA, CN, SI, JI, RA, HPI]
[0.0, 0, 0.0, 0, 0, 0, 0.0, 0]
[0.0, 0, 0.0, 0, 0, 0, 0.0, 0]
